<div style="border:1px solid black; padding:10px 10px;">
    <strong>CIVIL-321 "Modélisation Numérique des Solides et Structures"</strong><br/><br/>
    <span style="text-decoration:underline;font-weight:bold;">Comment utiliser ce Jupyter Notebook?
    </span><br/><br/>
    Ce <strong>Notebook</strong> est constitué de cellules de texte et de cellule de code. Les cellules de codes doivent être  <strong>executées</strong> pour voir le résultat du programme. Certaines cellules doivent être remplies par vos soins. Pour exécuter une cellule, cliquez dessus simplement et ensuite cliquez sur le bouton "play" (<span style="font: bold 12px/30px Arial, serif;">&#9658;</span>) dans la barre de menu au dessus du notebook. Vous pouvez aussi taper la combinaison de touches <code>shift + enter</code>. Il est important d'éxécuter les cellules de code en respectant leur ordre d'arrivée dans le notebook.
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from plot import *

# Formulation iso-paramétrique de l'élément plan quad4

Pour rappel, les éléments iso-paramétrique nous permettent de calculer la matrice de raideur en fonction de l'élément de référence (s, t) en non pas de chaque cas particulier d'élément (x, y).

La démarche est la suivante :
 - choix de l'élément
 - choix des fonctions d'interpolation $[N(s)]$
 - trouver la matrice $[B(s)]$
 - trouver la matrice de raideur $$[K] = \iint [B(s,t)]^T [D] [B(s,t)] det(J(s,t)) ds dt$$
     - $J$ est la matrice *Jacobienne*
 - trouver les forces consistantes

## Choix des fonctions d'interpolation $[N(s)]$

Deux méthodes s'offrent à nous:
- Méthode directe: les fonctions d'interpolation sont déterminées une par une grâce à la géométrie.
- Méthode indirecte par inversion de la matrice des monômes.

Méthode directe: 
- $N_1$ doit s'annuler sur les segments 2-3 et 3-4.
- les équations des droites sont:
    - $s = 1$ sur 2-3  et $t = 1$ sur 3-4
- La normalisation N_1(-1, -1) = 1 impose:
    - $N_1(s,t) = \frac{1}{4} (1-s)(1-t)$

Voici l'élément quad4 de référence:

<img src="Images/quad4-reference.svg">

---
#### Question : 

Déduire sur le modèle précédent les fonctions d'interpolation $N_2(s,t)$, $N_3(s,t)$, et $N_4(s,t)$.

---

 **Solution:**

 ---



- $N_1(s,t) = \frac{1}{4} (1-s)(1-t)$
- $N_2(s,t) = \frac{1}{4} (1+s)(1-t)$
- $N_3(s,t) = \frac{1}{4} (1+s)(1+t)$
- $N_4(s,t) = \frac{1}{4} (1-s)(1+t)$

Avec les fonctions d'interpolation $N_i$, on définit $u(s,t)$ et $v(s,t)$:

$$\left\{ 
\begin{array}{c} 
u(s,t) \\ 
v(s,t) 
\end{array} 
\right\} = 
\begin{bmatrix}
     N_1(s, t) & 0 & \ldots & N_4(s, t) & 0  \\
     0 & N_1(s, t) & \ldots & 0 & N_4(s, t) \\
\end{bmatrix}
\left\{ 
\begin{array}{c} 
u_1 \\ v_1 \\ \vdots \\ u_4 \\ v_4
\end{array} 
\right\}
$$

Mais aussi **le champ de coordonnées** $x(s,t)$ et $y(s,t)$:

$$\left\{ 
\begin{array}{c} 
x(s,t) \\ 
y(s,t) 
\end{array} 
\right\} = 
\begin{bmatrix}
     N_1(s, t) & 0 & \ldots & N_4(s, t) & 0  \\
     0 & N_1(s, t) & \ldots & 0 & N_4(s, t) \\
\end{bmatrix}
\left\{ 
\begin{array}{c} 
x_1 \\ y_1 \\ \vdots \\ x_4 \\ y_4
\end{array} 
\right\}
$$

iso-paramétrique $\equiv$ même paramètrisation $\equiv$ même espace naturel

## Trouver la matrice $[B(s)]$

La matrice $[B]$ permet d'obtenir le tenseur des petites déformations $\epsilon$.


$$[B] \{u\} = \left\{
\begin{array}{c}
\epsilon_{xx} \\
\epsilon_{yy} \\
2 \epsilon_{xy}
\end{array}
\right\}$$

*Rappel: en 1D iso-paramétrique nous avions* $[B(s)] = \frac{1}{J(s)}[N'(s)]$

On cherche donc une forme comparable en partant de:

$$ \left\{
\begin{array}{c}
     \epsilon_{xx} \\ \epsilon_{yy} \\ 2\epsilon_{xy}
\end{array}
\right\} = 
\left\{
\begin{array}{c}
     \frac{\partial u(x, y)}{\partial x} \\ \frac{\partial v(x, y)}{\partial y} \\ \frac{\partial u(x, y)}{\partial y} + \frac{\partial v(x, y)}{\partial x}
\end{array}
\right\}
= \underbrace{\begin{bmatrix}
1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 1 & 1 & 0 
\end{bmatrix}}_{A} 
\left\{ \begin{array}{c}
     \frac{\partial u(x, y)}{\partial x} \\ \frac{\partial u(x, y)}{\partial y} \\ \frac{\partial v(x, y)}{\partial x} \\ \frac{\partial v(x, y)}{\partial y}
\end{array} \right\}
$$

Dans un premier temps, intéressons nous aux termes diagonaux des déformations de $u$. 

La règle de dérivation en chaîne nous donne:

$$\frac{\partial \color{green}{u}(s,t)}{\partial \color{blue}{s}} = \frac{\partial \color{green}{u}(x,y)}{\partial \color{red}{x}} \frac{\partial \color{red}{x}(s, t)}{\partial \color{blue}{s}} + \frac{\partial \color{green}{u}(x,y)}{\partial \color{red}{y}} \frac{\partial \color{red}{y}(s,t)}{\partial \color{blue}{s}}$$

Dans la suite on notera $\nabla$ le gradient dans l'espace des coordonnées réelles ($x, y$) et 
$\tilde{\nabla}$ dans l'espace des coordonnées naturelles ($s, t$).

---

#### Question :
À partir des deux relations trouvées ci-dessus, exprimer la matrice Jacobienne $J(s,t)$ en fonction des gradients x et y par rapport aux variables $(s, t)$.

Cette matrice vérifie: 

$$\tilde \nabla u(s, t) = [J(s, t)] \nabla u(x, y)$$

---

 **Solution:**

 ---


$$\tilde \nabla u(s, t) = \left\{ \begin{array}{c}
\frac{\partial u(s,t)}{\partial s}  \\ 
\frac{\partial u(s,t)}{\partial t} 
\end{array} \right\} = 
\underbrace{\begin{bmatrix} \frac{\partial x(s,t)}{\partial s} & \frac{\partial y(s,t)}{\partial s} \\ \frac{\partial x(s,t)}{\partial t} & \frac{\partial y(s,t)}{\partial t}\end{bmatrix}}_{J(s,t)}
\left\{ \begin{array}{c}
\frac{\partial u(x, y)}{\partial x} \\ 
\frac{\partial u(x, y)}{\partial y}
\end{array} \right\} = [J(s, t)] \{\nabla u(s,t)\}$$

*Remarque: la jacobienne est une matrice carrée de taille la dimension spatiale de notre problème. En 3D c'est donc une* $3\times3$

Les gradients en coordonnées réelles s'expriment donc:
$$ \nabla u(x, y) = \begin{bmatrix} J(s, t) \end{bmatrix}^{-1} 
\tilde \nabla u(s,t) $$

et 

$$ \nabla v(x, y) = \begin{bmatrix} J(s, t) \end{bmatrix}^{-1} 
\tilde \nabla v(s,t) $$

Ainsi : 
$$ \left\{ \begin{array}{c}
 \nabla u(x, y) \\ 
 \nabla v(x, y) 
\end{array} \right\} = \begin{bmatrix} J(s, t) & 0 \\ 0 & J(s, t) \end{bmatrix}^{-1} \left\{ \begin{array}{c}
\tilde \nabla u(s,t)  \\ 
\tilde \nabla v(s,t)
\end{array} \right\} $$

En regroupant les termes on obtient finalement : 
$$ \left\{
\begin{array}{c}
     \epsilon_{xx} \\ \epsilon_{yy} \\ 2\epsilon_{xy}
\end{array}
\right\} 
= \begin{bmatrix}
1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 1 & 1 & 0 
\end{bmatrix}
\begin{bmatrix} J(s, t) & 0 \\ 0 & J(s, t) \end{bmatrix}^{-1} \left\{ \begin{array}{c}
\tilde\nabla u(s,t)  \\ 
\tilde \nabla v(s,t)
\end{array} \right\}
$$

Nous avons donc 5 termes à calculer:

1. $\tilde \nabla u \equiv \frac{\partial}{\partial s} u(s,t)$ et $\frac{\partial}{\partial t} u(s,t)$
2. $\tilde \nabla v \equiv \frac{\partial}{\partial s} v(s,t)$ et $\frac{\partial}{\partial t} v(s,t)$
3. $[J(s,t)]^{-1}$

## Calcul des gradients

Les gradients en espace naturels se calculent en dérivant par rapport à $s$ et $t$ le champ de déplacement:

$$\tilde \nabla u(s,t) \equiv \tilde \nabla [N(s, t)]\{d\}$$

---

#### Question : 
Explicitez tous les termes derrière cette equation.
Calculer ces coefficients pour le cas du Q4.

Pour rappel :
$$\left\{ 
\begin{array}{c} 
u(s,t) \\ 
v(s,t) 
\end{array} 
\right\} = 
\begin{bmatrix}
     N_1(s, t) & 0 & \ldots & N_4(s, t) & 0  \\
     0 & N_1(s, t) & \ldots & 0 & N_4(s, t) \\
\end{bmatrix}
\left\{ 
\begin{array}{c} 
u_1 \\ v_1 \\ \vdots \\ u_4 \\ v_4
\end{array} 
\right\}
$$

et
$$\begin{align}
N_1(s,t) &= \frac{1}{4} (1-s)(1-t) \qquad &N_2(s,t) &= \frac{1}{4} (1+s)(1-t)\\
N_3(s,t) &= \frac{1}{4} (1+s)(1+t) \qquad &N_4(s,t) &= \frac{1}{4} (1-s)(1+t)\\
\end{align}
$$

---

 **Solution:**

 ---



$$\left\{ 
\begin{array}{c} 
\frac{\partial}{\partial s} u(s,t) \\ 
\frac{\partial}{\partial t} u(s,t) 
\end{array} 
\right\} = 
\begin{bmatrix}
     \frac{\partial}{\partial s} N_1(s, t) & 0 & \ldots & \frac{\partial}{\partial s} N_4(s, t) & 0  \\
     \frac{\partial}{\partial t} N_1(s, t) & 0 & \ldots & \frac{\partial}{\partial t} N_4(s, t) & 0\\
\end{bmatrix}
\left\{ 
\begin{array}{c} 
u_1 \\ v_1 \\ \vdots \\ u_4 \\ v_4
\end{array} 
\right\}
$$

$$\tilde \nabla u(s, t) = \left\{ 
\begin{array}{c} 
\frac{\partial}{\partial s} u(s,t) \\ 
\frac{\partial}{\partial t} u(s,t) 
\end{array} 
\right\} = \frac{1}{4} 
\begin{bmatrix}
t-1 & 0 & -t+1 & 0 & t+1 & 0 & -t-1 & 0 \\
s-1 & 0 & -s-1 & 0 & s+1 & 0 & -s+1 & 0
\end{bmatrix}
\left\{ 
\begin{array}{c} 
u_1 \\ v_1 \\ \vdots \\ u_4 \\ v_4
\end{array} 
\right\}
$$

En considérant les dérivées portant sur $v(s, t)$, on trouve la relation complète :


$$
\left\{ 
\begin{array}{c} 
\tilde \nabla u(s,t) \\ 
\tilde \nabla v(s,t)
\end{array} 
\right\} = 
\left\{ 
\begin{array}{c} 
\frac{\partial}{\partial s} u(s,t) \\ 
\frac{\partial}{\partial t} u(s,t) \\
\frac{\partial}{\partial s} v(s,t) \\
\frac{\partial}{\partial t} v(s,t) \\ 
\end{array} 
\right\} = \underbrace{
\begin{bmatrix}
\frac{\partial}{\partial s}N_1(s,t) & 0 & \ldots & \frac{\partial}{\partial s}N_4(s,t) & 0 \\
\frac{\partial}{\partial t}N_1(s,t) & 0 & \ldots & \frac{\partial}{\partial t}N_4(s,t) & 0 \\
0 & \frac{\partial}{\partial s}N_1(s,t) & \ldots & 0 & \frac{\partial}{\partial s}N_4(s,t)\\
0 & \frac{\partial}{\partial t}N_1(s,t) & \ldots & 0 & \frac{\partial}{\partial t}N_4(s,t)\\
\end{bmatrix}}_{C(s,t)}
\left\{ 
\begin{array}{c} 
u_1 \\ v_1 \\ \vdots \\ u_4 \\ v_4
\end{array} 
\right\}
$$

La forme compacte d'un gradient dans l'espace naturel est donc:

$$\left\{ 
\begin{array}{c} 
\tilde \nabla u(s,t) \\ 
\tilde \nabla v(s,t)
\end{array} 
\right\} = [C(s, t)]\left\{ 
\begin{array}{c} 
u_1 \\ v_1 \\ \vdots \\ u_4 \\ v_4
\end{array} 
\right\}$$

Pour notre Q4: 

$$C(s,t) = \frac{1}{4} 
\begin{bmatrix}
t-1 & 0 & -t+1 & 0 & t+1 & 0 & -t-1 & 0 \\
s-1 & 0 & -s-1 & 0 & s+1 & 0 & -s+1 & 0 \\
0 & t-1 & 0 & -t+1 & 0 & t+1 & 0 & -t-1 \\
0 & s-1 & 0 & -s-1 & 0 & s+1 & 0 & -s+1
\end{bmatrix}$$

**Remarque: C(s,t) ne dépend pas des coordonnées réelles mais seulement des coordonnées naturelles**

On obtient donc une définition de $[B]$:
$$ \left\{
\begin{array}{c}
     \epsilon_{xx} \\ \epsilon_{yy} \\ 2\epsilon_{xy}
\end{array}
\right\} 
= \underbrace{\begin{bmatrix}
1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 1 & 1 & 0 
\end{bmatrix}
\begin{bmatrix} J(s, t) & 0 \\ 0 & J(s, t) \end{bmatrix}^{-1} 
\begin{bmatrix} C(s, t) \end{bmatrix}}_{B(s, t)}
\left\{ 
\begin{array}{c} 
u_1 \\ v_1 \\ \vdots \\ u_4 \\ v_4
\end{array} 
\right\}
$$

## Matrice Jacobienne 

La matrice Jacobienne étant:


$$\begin{bmatrix} J(s,t) \end{bmatrix}
= \begin{bmatrix} \frac{\partial x(s,t)}{\partial s} & \frac{\partial y(s,t)}{\partial s} \\ \frac{\partial x(s,t)}{\partial t} & \frac{\partial y(s,t)}{\partial t}\end{bmatrix} = 
\begin{bmatrix} 
\tilde \nabla x(s, t) & \tilde \nabla y(s,t )
\end{bmatrix}
$$

On va utiliser l'interpolation des coordonnées:
$$\left\{ 
\begin{array}{c} 
x(s,t) \\ 
y(s,t) 
\end{array} 
\right\} = [N(s, t)]\{d\} = 
\begin{bmatrix}
     N_1(s, t) & 0 & \ldots & N_4(s, t) & 0  \\
     0 & N_1(s, t) & \ldots & 0 & N_4(s, t) \\
\end{bmatrix}
\left\{ 
\begin{array}{c} 
x_1 \\ y_1 \\ \vdots \\ x_4 \\ y_4
\end{array} 
\right\}
$$

Par le principe iso-paramétrique on obtient un résultat similaire à celui des gradients de déplacements:


$$\left\{ 
\begin{array}{c} 
\tilde\nabla x(s,t) \\ 
\tilde\nabla y(s,t) 
\end{array} 
\right\} =  [C(s, t)]\left\{ 
\begin{array}{c} 
x_1 \\ y_1 \\ \vdots \\ x_4 \\ y_4
\end{array} 
\right\}$$

Ce qui donne pour notre Q4:

$$\left\{ 
\begin{array}{c} 
\tilde\nabla x(s,t) \\ 
\tilde\nabla y(s,t) 
\end{array} 
\right\} =
\frac{1}{4} 
\begin{bmatrix}
t-1 & 0 & -t+1 & 0 & t+1 & 0 & -t-1 & 0 \\
s-1 & 0 & -s-1 & 0 & s+1 & 0 & -s+1 & 0 \\
0 & t-1 & 0 & -t+1 & 0 & t+1 & 0 & -t-1 \\
0 & s-1 & 0 & -s-1 & 0 & s+1 & 0 & -s+1
\end{bmatrix} 
\left\{ 
\begin{array}{c} 
x_1 \\ y_1 \\ \vdots \\ x_4 \\ y_4
\end{array} 
\right\}
$$

Ainsi, en replacant les termes dans la matrice Jacobienne on obtient:

$$ \begin{align}
\boldsymbol{J} &= \begin{bmatrix} 
\frac{\partial}{\partial s} x(s, t) & \frac{\partial}{\partial s} y(s, t) \\
\frac{\partial}{\partial t} x(s, t) & \frac{\partial}{\partial t} y(s, t)
\end{bmatrix} \\
&= \frac{1}{4}
\begin{bmatrix}
x_1(t-1)+x_2(-t+1)+x_3(t+1)+x_4(-t-1) & y_1(t-1)+y_2(-t+1)+y_3(t+1)+y_4(-t-1) \\
x_1(s-1)+x_2(-s-1)+x_3(s+1)+x_4(-s+1) & y_1(s-1)+y_2(-s-1)+y_3(s+1)+y_4(-s+1)
\end{bmatrix}
\end{align}
$$

---

#### Question :

Prenons le cas particuliers d'un Q4 de même taille que le Q4 naturel de référence du quad4. Par exemple avec les coordonnées des noeuds suivantes:

- nœud 1 : x = 0 et y = 0
- nœud 2 : x = 2 et y = 0
- nœud 3 : x = 2 et y = 2
- nœud 4 : x = 0 et y = 2

Quelle est la matrice Jacobienne associée à cette structure ?

In [ ]:
# implémentation de la formule trouvée plus haut
def J(s, t, x):
    x_1, y_1 = x[0, :]
    x_2, y_2 = x[1, :]
    x_3, y_3 = x[2, :]
    x_4, y_4 = x[3, :]
    
    J = Matrix([
        [x_1*(t-1)+x_2*(-t+1)+x_3*(t+1)+x_4*(-t-1) , y_1*(t-1)+y_2*(-t+1)+y_3*(t+1)+y_4*(-t-1)],
        [x_1*(s-1)+x_2*(-s-1)+x_3*(s+1)+x_4*(-s+1) , y_1*(s-1)+y_2*(-s-1)+y_3*(s+1)+y_4*(-s+1)]])
    J *= Rational(1, 4)
    return simplify(J)
    
    
noeuds = np.array([
    [0, 0],
    [2, 0],
    [2, 2],
    [0, 2]])


s, t = symbols('s t')
plot_matrix(J(s, t, noeuds), 'J')

---

#### Question :

Que se passe-t-il si chaque coordonnée est multipliée par deux ? Quelle est votre interprétation physique ?

In [ ]:
###########
# Solution:
##########


noeuds_scaled = noeuds*2

plot_matrix(J(s, t, noeuds_scaled), 'J')

---

#### Question :

Que se passe-t-il si les coordonnées subissent une rotation ? Quelle est votre interprétation physique ?

In [ ]:
###########
# Solution:
##########


theta = Symbol(r'\theta')
rotation_matrix = np.array([[cos(theta), sin(theta)],
                          [-sin(theta), cos(theta)]])

noeuds_rotated = Matrix([
    rotation_matrix@noeuds[0, :],
    rotation_matrix@noeuds[1, :],
    rotation_matrix@noeuds[2, :],
    rotation_matrix@noeuds[3, :]
])



noeuds_rotated
plot_matrix(J(s, t, noeuds_rotated), 'J')

---

 **Solution:**

 ---



Si on applique un rotation au triangle, alors on procède à un changement de base entre naturel et réel. Avec cette définition:

$$\tilde \nabla u(s, t) = [J(s, t)] \{\nabla u(s,t)\}$$

il est logique de trouver que $J$ soit l'inverse(et donc sa transposée) de la matrice de rotation base naturelle $\rightarrow$ base réelle

---

#### Question :

Quelle est votre interprétation physique de $det(J)$ ?

---

 **Solution:**

 ---



C'est le ratio de volume/surface/longueur entre l'élément réel et l'élément naturel

## Trouver la matrice de raideur $[k]$

Dans l'espace des coordonnées naturelles, la matrice de raideur prend la forme donnée par changement de variable généralisée aux dimensions supérieures:

$$[k] = \iint [B(s,t)]^T [D] [B(s,t)] det(J(s,t)) ds dt$$

Une telle intégration peut vite s'avérer difficile à la main. Des méthodes numériques sont nettement préférable, par exemple à l'aide de la quadrature de Gauss.

### Algorithme général pour l'assemblage de la matrice de rigidité $K$ pour les éléments iso-paramétriques

Ceci doit être fait via deux boucles imbriquées:

1. une sur les éléments
2. une sur les points de quadratures

<img src="Images/algo-iso-parametrique3.svg">

In [ ]:
# par exemple pour un mesh de Q4

def calculerMatriceRigiditeLocale(connectivite_element, coordonnees):
       
    quads = [[-1/sqrt(3), -1/sqrt(3)],
             [1/sqrt(3),  -1/sqrt(3)],
             [1/sqrt(3),  1/sqrt(3)],
             [-1/sqrt(3),  1/sqrt(3)]]
    
    weights = [1, 1, 1, 1]
    
    for x_q, w_q in zip(quads, weights):
        B = calculerB(x_q, coordonnees) # calcule B sur le point de quadrature x_q
        J = calculerJ(x_q, coordonnees) # calcule J sur le point de quadrature x_q
        detJ = np.linalg.det(J)         # calcule det(J) pour l'intégration
        D = calculerD()                 # calcule la loi de constitution D
        
        
        # somme la quadrature avec le poid w_q
        K_local = Klocal + w_q * B.T@D@B * detJ
        
def assemblerMatriceRigidite(connectivite, coordonnees):

    n_elem  = connectivite.shape[0]
    n_nodes = coordonnees.shape[0]
    numEq = calculerNumerosEquations(connectivite)

    K = np.zeros((n_nodes*2, n_nodes*2))
    for e in range(n_elem):
        # On récupère les degrés de liberté de l'élément e
        ddl = numEq[e, :]
        # On récupère les noeuds de l'élément e
        connectivite_element = connectivite[e, :]
        # On calcule la matrice de rigidite locale de l'élément e
        K_locale = calculerMatriceRigiditeLocale(connectivite_element, coordonnees)
        # On assemble
        for i, gi in enumerate(ddl):
            for j, gj in enumerate(ddl):
                K[gi, gj] += K_locale[i, j]
    return K

## Trouver les forces consistantes 

Dans l'espace des coordonnées naturelles, les forces consistantes s'expriment par :
$$ \{ f \} = \iint [N(s, t)]^T \{X_b(s, t)\} det(J(s,t)) ds dt$$

---

#### Question: 

Quel est l'algorithme iso-paramétrique pour le calcul des forces consistantes ?